# Prepare data

In [32]:
import pandas as pd

In [33]:
df = pd.read_csv('data/glue/MRPC/train.tsv', sep='\t', header=0, error_bad_lines=False)

b'Skipping line 93: expected 5 fields, saw 6\nSkipping line 590: expected 5 fields, saw 6\nSkipping line 778: expected 5 fields, saw 6\nSkipping line 790: expected 5 fields, saw 6\nSkipping line 882: expected 5 fields, saw 6\nSkipping line 1296: expected 5 fields, saw 6\nSkipping line 1324: expected 5 fields, saw 6\nSkipping line 1640: expected 5 fields, saw 6\nSkipping line 1759: expected 5 fields, saw 6\nSkipping line 1808: expected 5 fields, saw 6\nSkipping line 2010: expected 5 fields, saw 6\nSkipping line 2254: expected 5 fields, saw 6\nSkipping line 2268: expected 5 fields, saw 6\nSkipping line 2527: expected 5 fields, saw 6\nSkipping line 2602: expected 5 fields, saw 6\nSkipping line 2659: expected 5 fields, saw 6\nSkipping line 2929: expected 5 fields, saw 6\nSkipping line 3032: expected 5 fields, saw 6\nSkipping line 3144: expected 5 fields, saw 6\nSkipping line 3322: expected 5 fields, saw 6\nSkipping line 3563: expected 5 fields, saw 6\n'


In [34]:
df = df.dropna()

In [1]:
import csv

In [2]:
# Input data

lines = []
with open('data/bert/fine-tuned/MRPC/test.tsv', "r") as f:
    reader = csv.reader(f, delimiter="\t", quotechar=None)
    for line in reader:
        lines.append(line)

In [9]:
# Predictions

predictions = []
with open('data/bert/fine-tuned/MRPC/test_results.tsv', 'r') as f:
    reader = csv.reader(f, delimiter="\t", quotechar=None)
    for line in reader:
        predictions.append(line)

In [4]:
sentence_pairs = [' ||| '.join([line[3], line[4]]) for line in lines[1:]]

In [6]:
sentence_pairs[0]

'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence . ||| Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .'

# Store embeddings to db

In [7]:
from bert_serving.client import BertClient

bc = BertClient()

In [8]:
from pymongo import MongoClient

client = MongoClient()

db = client['glvis_db']
val_collection = db['bert_mrpc']

In [10]:
reduce_mean = bc.encode(sentence_pairs)

In [11]:
reduce_mean.shape

(5801, 768)

In [13]:
cls_token = bc.encode(sentence_pairs)

In [14]:
cls_token.shape

(5801, 768)

In [16]:
len(sentence_pairs)

5801

In [17]:
len(predictions)

5801

In [26]:
db_entries = [{
    'sentence': pair,
    'reduce_mean': reduce_mean[i].tolist(),
    'cls_token': cls_token[i].tolist(),
    'prediction': [float(predictions[i][0]), float(predictions[i][1])],
    'id': i
} for i, pair in enumerate(sentence_pairs)]

In [27]:
len(db_entries)

5801

In [28]:
val_collection.drop()

In [29]:
val_collection.insert_many(db_entries)

# Query results from db

In [3]:
reduce_mean = []
cls_token = []

for cur in val_collection.find():
    reduce_mean.append(cur['reduce_mean'])
    cls_token.append(cur['cls_token'])

In [14]:
import numpy as np

In [15]:
reduce_mean = np.array(reduce_mean)
cls_token = np.array(cls_token)

# Visualizing results

In [6]:
from clustergrammer_widget import *
net = Network(clustergrammer_widget)

In [ ]:
emb_df = pd.DataFrame(cls_token[100:500, 500:800])

# load DataFrame
net.load_df(emb_df)

# cluster using default parameters
net.cluster(enrichrgram=False)

# make interactive widget
net.widget()

# View prediction results

In [1]:
import pandas as pd

In [19]:
import csv

In [139]:
# Use csv to read
lines = []

with open('data/glue/MRPC/train.tsv', "r") as f:
    reader = csv.reader(f, delimiter="\t", quotechar=None)
    for line in reader:
        lines.append(line[1:])

with open('data/glue/MRPC/dev.tsv', "r") as f:
    reader = csv.reader(f, delimiter="\t", quotechar=None)
    for i, line in enumerate(reader):
        if i == 0: continue
        lines.append(line[1:])

with open('data/glue/MRPC/test.tsv', "r") as f:
    reader = csv.reader(f, delimiter="\t", quotechar=None)
    for i, line in enumerate(reader):
        if i == 0: continue
        lines.append(line[1:])

In [141]:
len(lines)

5802

In [140]:
with open('data/bert/fine-tuned/MRPC/test.tsv', 'w', newline='') as f:
    writer = csv.writer(f, delimiter='\t', quotechar=None)
    for i, line in enumerate(lines):
        if i == 0:
            writer.writerow(['index'] + line)
            continue
        writer.writerow([i] + line)

In [142]:
predictions = []
with open('data/bert/fine-tuned/MRPC/test_results.tsv', 'r') as f:
    reader = csv.reader(f, delimiter="\t", quotechar=None)
    for line in reader:
        predictions.append(line)

In [143]:
len(predictions)

5801

In [121]:
len(predictions)

5802